In [1]:
import xarray as xr

In [2]:
import distributed

client = distributed.Client("tcp://scheduler:8786")
client

/venv/lib/python3.12/site-packages/distributed/client.py:1591: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.1.1  | 3.1.2     | 3.1.2   |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://10.2.0.131:8786' processes=1 threads=1, memory=14.90 GiB>

In [3]:
era5_land_hourly = xr.open_dataset(
    "s3://edc-era5/reanalysis-era5-land-no-antartica-v0.zarr",
    engine="zarr",
    chunks={"latitude": 250, "longitude": 250},
).drop_vars(["depthBelowLandLayer", "number", "surface"])
era5_land_hourly

/venv/lib/python3.12/site-packages/xarray_sentinel/esa_safe.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/tmp/ipykernel_4419/2177117717.py:1: UserWarning: The specified chunks separate the stored chunks along dimension "latitude" starting at index 250. This could degrade performance. Instead, consider rechunking after loading.
  era5_land_hourly = xr.open_dataset(
/tmp/ipykernel_4419/2177117717.py:1: UserWarning: The specified chunks separate the stored chunks along dimension "longitude" starting at index 250. This could degrade performance. Instead, consider rechunking after loading.
  era5_land_hourly = xr.open_dataset(


<xarray.Dataset> Size: 705TB
Dimensions:     (valid_time: 664752, latitude: 1472, longitude: 3600)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 5MB 1950-01-01 ... 2025-10-31T23:...
  * latitude    (latitude) float64 12kB 90.0 89.9 89.8 ... -56.9 -57.0 -57.1
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
Data variables: (12/50)
    asn         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    e           (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    es          (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    evabs       (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    evaow       (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    ...          ...
    swvl4       (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    t2m         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    tp          (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    tsn         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 14TB dask.array<chunksize=(2880, 250, 250), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2024-10-29T11:44 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [13]:
data_var_hourly = era5_land_hourly.ssrd

data_var_daily = data_var_hourly[::24].shift(valid_time=-1)
data_var_daily

<xarray.DataArray 'ssrd' (valid_time: 27698, latitude: 1472, longitude: 3600)> Size: 587GB
dask.array<rechunk-merge, shape=(27698, 1472, 3600), dtype=float32, chunksize=(120, 250, 250), chunktype=numpy.ndarray>
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 222kB 1950-01-01 ... 2025-10-31
  * latitude    (latitude) float64 12kB 90.0 89.9 89.8 ... -56.9 -57.0 -57.1
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
Attributes: (12/30)
    GRIB_NV:                                  0
    GRIB_Nx:                                  3600
    GRIB_Ny:                                  1472
    GRIB_cfName:                              surface_downwelling_shortwave_f...
    GRIB_cfVarName:                           ssrd
    GRIB_dataType:                            fc
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_typeOfLevel:                         surface
    GRIB_units:                               J m**-2
    long_name:                                Surface short-wave (solar) radi...
    standard_name:                            surface_downwelling_shortwave_f...
    units:                                    J m**-2

In [14]:
data_var_daily = data_var_daily.chunk(valid_time=60)
data_var_daily

<xarray.DataArray 'ssrd' (valid_time: 27698, latitude: 1472, longitude: 3600)> Size: 587GB
dask.array<rechunk-merge, shape=(27698, 1472, 3600), dtype=float32, chunksize=(60, 250, 250), chunktype=numpy.ndarray>
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 222kB 1950-01-01 ... 2025-10-31
  * latitude    (latitude) float64 12kB 90.0 89.9 89.8 ... -56.9 -57.0 -57.1
  * longitude   (longitude) float64 29kB 0.0 0.1 0.2 0.3 ... 359.7 359.8 359.9
Attributes: (12/30)
    GRIB_NV:                                  0
    GRIB_Nx:                                  3600
    GRIB_Ny:                                  1472
    GRIB_cfName:                              surface_downwelling_shortwave_f...
    GRIB_cfVarName:                           ssrd
    GRIB_dataType:                            fc
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_typeOfLevel:                         surface
    GRIB_units:                               J m**-2
    long_name:                                Surface short-wave (solar) radi...
    standard_name:                            surface_downwelling_shortwave_f...
    units:                                    J m**-2

In [15]:
data_var_daily.latitude.encoding.clear()
data_var_daily.longitude.encoding.clear()

In [16]:
%%time

import zarr

data_var_daily.to_zarr(
    "s3://edc-era5/era5-land-daily-utc-v0.zarr",
    encoding={
        "ssrd": {
            "compressors": [zarr.codecs.BloscCodec(cname="zstd", clevel=5, shuffle=zarr.codecs.BloscShuffle.shuffle)],
            "filters": [zarr.codecs.BitRound(keepbits=10)],
        },
    },
    zarr_format=3,
    mode="a",
    write_empty_chunks=False,
)


/venv/lib/python3.12/site-packages/zarr/codecs/numcodecs/_codecs.py:161: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)
/venv/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
/venv/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 14.07 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 4.89 s, sys: 783 ms, total: 5.68 s
Wall time: 1h 8min 10s


In [14]:
import math

fractional_accuracy = 0.05 / 1000

bitkeep = math.ceil(-math.log2(fractional_accuracy)) - 2
bitkeep

13